# Graph Neural Network for Node Classification

In [1]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import math
import numpy as np
import scipy.sparse as sp


## Model Definition

### Graph Convolution Operation

In [2]:
class GraphConvolution(nn.Module):
    def __init__(self, feature_numbers, out_numbers, bias=False) -> None:
        super().__init__()
        self.bias = bias
        self.w = Parameter(torch.FloatTensor(feature_numbers, out_numbers))
        if bias:
            self.b = Parameter(torch.FloatTensor(out_numbers))
        self.reset_parameters()
        
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.w.size(1))
        self.w.data.uniform_(-stdv, stdv)
        if self.bias is not False:
            self.b.data.uniform_(-stdv, stdv)
            
    def forward(self, x, adj):
        support = torch.mm(x, self.w)
        out = torch.spmm(adj, support)
        if self.bias:
            out = out + self.b

        return out

### Node Classification Model

In [3]:
class NodeClassificationGCNN(nn.Module):

    def __init__(self, feature_num, node_representation_dim, nclass, droupout=0.2, bias=False) -> None:
        super().__init__()
        self.gconv1 = GraphConvolution(feature_num, node_representation_dim, bias)
        self.gconv2 = GraphConvolution(node_representation_dim, nclass, bias)
        self.dropout = droupout

    def forward(self, x, adj):
        x = F.relu(self.gconv1(x, adj))
        x = F.dropout(x, self.dropout, self.training)
        x = F.relu(self.gconv2(x, adj))
        return F.log_softmax(x, dim=1)

## Data Loading

In [4]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def load_data(path="../data/cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

adj, features, labels, idx_train, idx_val, idx_test=load_data(path="../data/cora/", dataset="cora")

Loading cora dataset...


/tmp/ipykernel_205002/3819785820.py:25: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:651.)
  return torch.sparse.FloatTensor(indices, values, shape)


## Model Training

In [5]:
model = NodeClassificationGCNN(features.shape[1], 256, np.max(labels.detach().numpy())+1)

In [6]:
def accuracy(out,label):
    oneHotCodded = out.max(1)[1].type_as(label)
    return oneHotCodded.eq(label).double().sum()/len(label)

In [7]:
epochs=100
optimizer = optim.Adam(model.parameters(),lr=0.01)

for epoch in range(epochs):
    model.train()
    train_labels=labels[idx_train]
    val_labels=labels[idx_val]
    
    
    optimizer.zero_grad()
    output = model(features, adj)
    loss=F.nll_loss(output[idx_train],train_labels)
    print(f"Training epoch {epoch} ; accuracy: {accuracy(output[idx_train],train_labels)}; loss: {loss.item()}")
    loss.backward()
    optimizer.step()
    
    model.eval()
    output = model(features, adj)
    loss=F.nll_loss(output[idx_val],val_labels)
    print(f"Validation epoch {epoch} ; accuracy: {accuracy(output[idx_val],val_labels)}; loss: {loss.item()}")
    

Training epoch 0 ; accuracy: 0.17857142857142858; loss: 1.9436419010162354
Validation epoch 0 ; accuracy: 0.6066666666666667; loss: 1.8875856399536133
Training epoch 1 ; accuracy: 0.6571428571428571; loss: 1.8710966110229492
Validation epoch 1 ; accuracy: 0.6233333333333333; loss: 1.81190025806427
Training epoch 2 ; accuracy: 0.7; loss: 1.7783282995224
Validation epoch 2 ; accuracy: 0.66; loss: 1.7257922887802124
Training epoch 3 ; accuracy: 0.7571428571428571; loss: 1.667269229888916
Validation epoch 3 ; accuracy: 0.6833333333333333; loss: 1.634028673171997
Training epoch 4 ; accuracy: 0.8071428571428572; loss: 1.548073410987854
Validation epoch 4 ; accuracy: 0.7033333333333334; loss: 1.538655161857605
Training epoch 5 ; accuracy: 0.85; loss: 1.421162724494934
Validation epoch 5 ; accuracy: 0.7266666666666667; loss: 1.443504810333252
Training epoch 6 ; accuracy: 0.8571428571428571; loss: 1.2948830127716064
Validation epoch 6 ; accuracy: 0.7366666666666667; loss: 1.3469977378845215
Tra

## Model Evaluation

In [8]:
model.eval()
test_labels=labels[idx_test]
output = model(features, adj)
loss=F.nll_loss(output[idx_test],test_labels)
print(f"Test set ; accuracy: {accuracy(output[idx_test],test_labels)}; loss: {loss.item()}")

Test set ; accuracy: 0.8; loss: 0.6505059599876404
